<a href="https://colab.research.google.com/github/billgatos/IA25_P01_G03/blob/main/IA_P01_G03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-constraint

import constraint
from constraint import *
import random
from typing import Dict, List, Tuple, Any

# --- New Function: Data Importer ---

from google.colab import files

filepath = files.upload()



  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24061 sha256=8332f770e8a9188d52c5f2079e780673e4e94fe79bdb512f1cba3539ec231a29
  Stored in directory: /root/.cache/pip/wheels/c1/d2/3d/082849b61a9c6de02d4a7c8a402c224640f08d8a971307b92b
Successfully built python-constraint


Saving ClassTT_01_tiny.txt to ClassTT_01_tiny.txt


In [2]:

def load_timetable_data(filepath: str) -> Dict[str, Any]:
    """
    Reads and parses the timetable definition data from an external text file.

    Args:
        filepath: The path to the data file (e.g., 'ClassTT_01_tiny.txt').

    Returns:
        A dictionary containing parsed data structures.
    """
    data = {
        'CLASS_COURSES': {},
        'TEACHER_COURSES': {},
        'COURSE_ROOM_RESTRICTIONS': {},
        'TEACHER_UNAVAILABLE_SLOTS': {},
        'ONLINE_LESSONS': {},
        'COURSE_TEACHERS': {} # Calculated later
    }

    current_section = None

    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#head') or line.startswith('#olw'):
                    continue

                if line.startswith('#'):
                    current_section = line
                    continue

                parts = line.split()
                if not parts:
                    continue

                key = parts[0]
                values = parts[1:]

                if current_section == '#cc':
                    # (class, courses*)
                    data['CLASS_COURSES'][key] = values

                elif current_section == '#dsd':
                    # (teacher, courses*)
                    data['TEACHER_COURSES'][key] = values

                elif current_section == '#tr':
                    # (teacher, slots_unavailable*)
                    # Convert slot numbers to integers
                    data['TEACHER_UNAVAILABLE_SLOTS'][key] = [int(v) for v in values]

                elif current_section == '#rr':
                    # (course, room) - values[0] is the room
                    data['COURSE_ROOM_RESTRICTIONS'][key] = values[0]

                elif current_section == '#oc':
                    # (course, lesson_week_index) - values[0] is the lesson index
                    # Note: We expect the lesson index to be '1' or '2' for L1/L2
                    data['ONLINE_LESSONS'][key] = int(values[0])

    except FileNotFoundError:
        print(f"Error: File not found at {filepath}. Using default hardcoded data for execution.")
        # Fallback to hardcoded data (or raise error)
        return {
            'CLASS_COURSES': {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'], 't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'], 't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']},
            'TEACHER_COURSES': {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']},
            'TEACHER_UNAVAILABLE_SLOTS': {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]},
            'COURSE_ROOM_RESTRICTIONS': {'UC14': 'Lab01', 'UC22': 'Lab01'},
            'ONLINE_LESSONS': {'UC21': 2, 'UC31': 2},
            'COURSE_TEACHERS': {'UC11': 'jo', 'UC21': 'jo', 'UC22': 'jo', 'UC31': 'jo', 'UC12': 'mike', 'UC23': 'mike', 'UC32': 'mike', 'UC13': 'rob', 'UC14': 'rob', 'UC24': 'rob', 'UC33': 'rob', 'UC15': 'sue', 'UC25': 'sue', 'UC34': 'sue', 'UC35': 'sue'}
        }


    # Invert TEACHER_COURSES to COURSE_TEACHERS (essential for constraints)
    for teacher, courses in data['TEACHER_COURSES'].items():
        for course in courses:
            data['COURSE_TEACHERS'][course] = teacher

    return data

# --- Load Data and Define Global Constants ---

# NOTE: For this code to run successfully in a Jupyter environment,
# you'd need to ensure the file 'ClassTT_01_tiny.txt' is in the same directory.
DATA = load_timetable_data('ClassTT_01_tiny.txt')

# Time domain constants
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
BLOCKS = list(range(1, 21))  # 1 to 20 total blocks (5 days * 4 blocks/day)
ROOMS = ['R1', 'R2', 'R3', 'R4', 'Lab01', 'Online']

# Assign loaded data to easy-to-use variables
CLASS_COURSES = DATA['CLASS_COURSES']
COURSE_TEACHERS = DATA['COURSE_TEACHERS']
TEACHER_UNAVAILABLE_SLOTS = DATA['TEACHER_UNAVAILABLE_SLOTS']
COURSE_ROOM_RESTRICTIONS = DATA['COURSE_ROOM_RESTRICTIONS']
ONLINE_LESSONS = DATA['ONLINE_LESSONS']
TEACHER_COURSES = DATA['TEACHER_COURSES'] # Used in Constraint B

Fim da definição de variaveis e leitura de data set .